In [7]:
import numpy as np
import pandas as pd
# import bike_project as bp

In [83]:
urls = {
        'Ballard' : 'https://data.seattle.gov/resource/47yq-6ugv.json',
        'Capitol_Hill' : 'https://data.seattle.gov/resource/j4vh-b42a.json',
        'Central_2nd_Ave' : 'https://data.seattle.gov/resource/avwm-i8ym.json',
#         'Central_7th_Ave' : 'https://data.seattle.gov/resource/qfzg-zmyj.json',
#         'West_Seattle' : 'https://data.seattle.gov/resource/mefu-7eau.json',
#         'I90_lid' : 'https://data.seattle.gov/resource/u38e-ybnc.json',
#         'NE_Seattle' : 'https://data.seattle.gov/resource/2z5v-ecg8.json',
#         'Myrtle_Edwards' : 'https://data.seattle.gov/resource/4qej-qvrz.json'
        }
limit = 30000
offset = 0

with open('data/app_token.txt', 'r') as t_file: 
    app_token = t_file.read() 

shop_dict = pd.read_csv('data/bike_shop_tally.csv', header=None)
for k, v in shop_dict.items():
    print(k, shop_dict[k])
    
n_shops = 3
    

0 0            Ballard
1    Central_2nd_Ave
2    Central_7th_Ave
3       West_Seattle
4       Capitol_Hill
5         NE_Seattle
6     Myrtle_Edwards
7            I90_lid
Name: 0, dtype: object
1 0    3
1    5
2    6
3    3
4    2
5    6
6    3
7    5
Name: 1, dtype: int64


In [84]:
def read_bike_trips(path, location_name, num_shops, app_token, limit=50000, offset=50000):
    '''
    Reads an API accessible dataset given url path and other url call variables
    Parameters:
    -----------
    path (str): url path beginning with https:
    location_name (str): name to assign to the trip counts column
    token (str): app_token
    limit (int): max rows to request
    offset (int): starting point for gathering rows

    Returns:
    --------
    pandas dataframe
    '''
    df = pd.read_json( f'{path}?$limit={limit}&$offset={offset}&$$app_token={app_token}')
    
    # rename trip_count column to location_name
    df[location_name] = df.iloc[:,1]   
    
    # calculate date, month, year, dow, commuter (boolean)
    df['short_date'] = pd.DatetimeIndex(df['date']).date
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['dow'] = pd.DatetimeIndex(df['date']).dayofweek
    df['hour'] = pd.DatetimeIndex(df['date']).hour
    df['am_commuter'] = df['dow'].isin([7,1,2,3,4]) & (df['hour'].isin([5,6,7,8,9]))
    df[f'{location_name}_am_peak'] = np.where(df['am_commuter']==True, df[location_name], 0)
    df[f'{location_name}_other'] = np.where(df['am_commuter']==False, df[location_name], 0)
    
    # collapse table by date and create sum counts for commuter and (other-(2 x commuter))
    df_by_date = df.groupby(['short_date', 'month', 'year', 'dow']).agg(
                                        {f'{location_name}_am_peak':'sum',
                                         f'{location_name}_other':'sum'
                                          })
    
    # add in count of nearby bike shops
    df_by_date[f'{location_name}_bike_shops'] = num_shops
    print(df_by_date)
    

In [85]:
df = read_bike_trips(urls['Ballard'], 'Ballard', n_shops, app_token, limit, offset)
# for k, v in urls.items():
#     if k != 'Ballard':
#         df_ = df_.append(bp.read_bike_trips(v, k, n_shops, app_token, limit, offset))

                           Ballard_am_peak  Ballard_other  Ballard_bike_shops
short_date month year dow                                                    
2014-01-01 1     2014 2               15.0          369.0                   3
2014-01-02 1     2014 3               80.0          375.0                   3
2014-01-03 1     2014 4               86.0          491.0                   3
2014-01-04 1     2014 5                0.0          555.0                   3
2014-01-05 1     2014 6                0.0          488.0                   3
...                                    ...            ...                 ...
2017-05-30 5     2017 1                0.0            0.0                   3
2017-05-31 5     2017 2                0.0            0.0                   3
2017-06-01 6     2017 3                0.0            0.0                   3
2017-06-02 6     2017 4                0.0          100.0                   3
2017-06-03 6     2017 5                0.0          154.0       

In [12]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
bp.gen_date_time(df_)
df_

In [ ]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
df['dow'] = pd.DatetimeIndex(df['date']).dayofweek
df['hour'] = pd.DatetimeIndex(df['date']).hour
df['am_commuter'] = df['dow'].isin([7,1,2,3,4]) & (df['hour'].isin([5,6,7,8,9]))
am_commuter_group = (df[df['am_commuter'] == True])
am_commuter_group


In [ ]:
df.groupby(df[['date', 'commuter']])